# NOTE FROM BRIAN

I uploaded reg1.txt and reg2.txt to your lab space. Those files are the output from my reg1 and reg2 runs. Take a look at them if your output does not contain the results that you need. Good luck!

## Part 1 - Data Preparation and Exploration 

%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [1]:
#Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
import statsmodels.formula.api as sm 

In [2]:
# code to suppress scientific notation in pandas
pd.set_option('display.float_format', lambda x: '%.15f' % x) # '%.15f' controls the no of digits shown after the decimal point
# To reset, simply run the next line
# pd.reset_option('display.float_format')

In [3]:
# Import Shotlog_14_15 and Player_Stats Datasets

Shotlog_1415 = pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
Player_Stats_1415 = pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")

In [ ]:
display(Shotlog_1415)

In [ ]:
Player_Stats_1415.head()

In [ ]:
### Background Info on the dataset
Shotlog_1415.info()

In [ ]:
# Trying to know the number of games covered in the dataset
Shotlog_1415.game_id.nunique()

In [ ]:
# Trying to know number of players covered in this dataset
Shotlog_1415.shoot_player.nunique()

##### Descriptions of the datasets and selected variables
a)    In the dataset “Shotlog_14_15,” each observation represents an attempt of a shot. In the dataset “Player_Stats_14_15,” each observation represents a player.

b)   The “average_hit” variable in both dataframes indicate the average success rate of a player making a shot over the season. It is defined and calculated the same way in both dataframes.

c)    The variable “home_away” indicates whether the team that the player belongs to played at home or away.

d)    The variable “result” indicates whether the team that the player belongs to won or lost the game. The variable “final_margin” represents the difference in final score between the team the player belongs to and their opponent’s.

e)    The variable “shot_number” is the order of the shot the given player attempted at the given game.

f)    “game_clock” is the countdown clock for each quarter. The game clock starts at 12 minutes. “shot_clock” refers to the display of a countdown clock of the time within which the team possessing the ball must attempt a field goal. The shot clock starts at 24 seconds.

###### Convert the “date” variable to a date type variable and calculate summary statistics for the “shot_clock” variable.

In [4]:
Shotlog_1415['date'] =  pd.to_datetime(Shotlog_1415['date'])
Shotlog_1415['shot_clock'].describe()

count   122502.000000000000000
mean        12.453343618879691
std          5.763265011581127
min          0.000000000000000
25%          8.199999999999999
50%         12.300000000000001
75%         16.674999999999997
max         24.000000000000000
Name: shot_clock, dtype: float64

##### Create a lagged variable “lag_shot_hit” to indicate the result of the previous shot by the same player at the same game.
**Hint: In this dataset, the variable “match” may not be able to uniquely identify each game; you can use “game_id” instead. You can sort the data by shot number for each player to create the lagged variable.** 

In [ ]:
print(Shotlog_1415.columns.tolist())

In [ ]:
pd.set_option('display.max_columns', 100)
Shotlog_1415.head()

In [5]:
Shotlog_1415['lag_shot_hit']=Shotlog_1415.sort_values(by=['quarter','shot_number'],
                                            ascending=[True, True]).groupby(['shoot_player','game_id'])['current_shot_hit'].shift(1)
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,201572,6.600000000000000,1,2,al horford,201143,0.541258741258741,715,10,nan
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,201572,5.600000000000000,0,0,al horford,201143,0.541258741258741,715,10,1.000000000000000
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,201572,4.700000000000000,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,201572,5.800000000000000,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,201572,6.400000000000000,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000


###### Create a variable “error” to indicate the prediction error for each shot and a variable “lagerror” for the prediction error for the previous shot. The “error” variable is defined as the difference between the outcome of the current shot and the average success rate (“average_hit”) and the “lagerror” variable is defined as the difference between the outcome of the previous shot and the average success rate.

In [6]:
Shotlog_1415['error']= Shotlog_1415['current_shot_hit'] - Shotlog_1415['average_hit']
Shotlog_1415['lagerror'] = Shotlog_1415['lag_shot_hit'] - Shotlog_1415['average_hit']
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,1,2,al horford,201143,0.541258741258741,715,10,nan,0.458741258741259,nan
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,0,al horford,201143,0.541258741258741,715,10,1.000000000000000,-0.541258741258741,0.458741258741259
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741


###### Calculate summary statistics for the “error” and “lagerror” variables.

In [ ]:
Shotlog_1415[['error','lagerror']].describe()

## Part 2 - Conditional Probability and Autocorrelation

##### Instructions Manual For Testing Hot Hand
- Create a dummy variable “conse_shot” that indicates a player made consecutive shots. 

- Create a dataframe “Player_Prob_1415” for the probability of making the previous shot and the joint probability for making both the previous and current shots. Name the probability of making the previous shot “average_lag_hit” and the probability of making both shots “conse_shot_hit.” 

- In the “Player_Prob_1415” dataframe, calculate the conditional probability “conditional_prob” for a player to make a shot given that he made the previous shot. 

- Merge the “Player_Prob_1415” dataframe into the “Player_Stats_1415” dataframe.

- Calculate summary statistics for the probability for a player to make a shot (“average_hit”) and the conditional probability for a player to make a shot given that he made the previous one (“conditional_prob”) and the probability of players making consecutive shots (“conse_shot_hit”).

- Perform a t-test for the statistical significance on the difference between conditional probability and unconditional probability of making a shot.

- Calculate the first order autocorrelation coefficient on making a shot (correlation coefficient between making the current shot and the previous shot) for the entire shotlog dataset.

- Calculate the first order autocorrelation coefficient on making a shot for each player. Display the top ten players with the highest first order autocorrelation coefficient.

## Conditional Probability
We can first calculate the conditional probability of making a shot in the current period conditional on making the previous shot. 
$$Conditional \ Probability=\frac{Probability \ of \ Making \ Consecutive \ Shots}{Probability \ of \ Making \ Previous \ Shot}$$

We will need to create a variable that indicates a player made consecutive shots.

In [7]:
Shotlog_1415['conse_shot'] = np.where((Shotlog_1415['current_shot_hit']==1)&(Shotlog_1415['lag_shot_hit']==1), 1, 0) 
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror,conse_shot
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,2,al horford,201143,0.541258741258741,715,10,nan,0.458741258741259,nan,0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,al horford,201143,0.541258741258741,715,10,1.000000000000000,-0.541258741258741,0.458741258741259,0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741,0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741,0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,al horford,201143,0.541258741258741,715,10,0.000000000000000,-0.541258741258741,-0.541258741258741,0


As in the lectures, we can create a player level dataframe. **The average of the "conse_shot" variable (named as "conse_shot_hit") would be the joint probability of making current and previous shots.** We will also calculate **the average of "lag_shot_hit" to indicate the probability of making the previous shot.**

In [ ]:
Shotlog_1415.head()

In [8]:
Player_Prob_1415 = Shotlog_1415.groupby(['shoot_player'])['conse_shot','lag_shot_hit'].mean()
Player_Prob_1415 = Player_Prob_1415.reset_index()

Player_Prob_1415.rename(columns={'conse_shot':'conse_shot_hit','lag_shot_hit':'average_lag_hit'}, inplace=True)
Player_Prob_1415.head()

,shoot_player,conse_shot_hit,average_lag_hit
0,aaron brooks,0.153297682709447,0.418000000000000
1,aaron gordon,0.201923076923077,0.532467532467532
2,al farouq aminu,0.162790697674419,0.465686274509804
3,al horford,0.262937062937063,0.537993920972644
4,al jefferson,0.207500000000000,0.480000000000000


Also, as understood from the classroom, we calculate the conditional probability for each player
We can calculate the conditional probability by dividing the joint probability by the probability of making the previous shot.

In [9]:
Player_Prob_1415['conditional_prob'] = Player_Prob_1415['conse_shot_hit']/Player_Prob_1415['average_lag_hit']
Player_Prob_1415.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob
0,aaron brooks,0.153297682709447,0.418000000000000,0.366740867725951
1,aaron gordon,0.201923076923077,0.532467532467532,0.379221388367730
2,al farouq aminu,0.162790697674419,0.465686274509804,0.349571603427173
3,al horford,0.262937062937063,0.537993920972644,0.488736122634428
4,al jefferson,0.207500000000000,0.480000000000000,0.432291666666667


We can now merge the "Player_Prob_1415" data frame with the "Player_Stats_1415" data frame we created earlier to compare the conditional probability **(conditional_prob)** and the unconditional probability **(average_hit)**. 
##### If the two probabilities are the same, or almost the same, then we fail to find evidence that the making the current shot depends on making the previous shot.

In [10]:
Player_Stats_1415 = pd.merge(Player_Prob_1415, Player_Stats_1415, on=['shoot_player'])
Player_Stats_1415.head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
0,aaron brooks,0.153297682709447,0.418000000000000,0.366740867725951,0.415329768270945
1,aaron gordon,0.201923076923077,0.532467532467532,0.379221388367730,0.528846153846154
2,al farouq aminu,0.162790697674419,0.465686274509804,0.349571603427173,0.430232558139535
3,al horford,0.262937062937063,0.537993920972644,0.488736122634428,0.541258741258741
4,al jefferson,0.207500000000000,0.480000000000000,0.432291666666667,0.477500000000000
5,alan anderson,0.157270029673590,0.462365591397849,0.340142157201021,0.433234421364985
6,alan crabbe,0.138297872340426,0.523809523809524,0.264023210831721,0.425531914893617
7,alex len,0.247491638795987,0.539419087136929,0.458811422691021,0.528428093645485
8,alexis ajinca,0.284360189573460,0.598802395209581,0.474881516587678,0.597156398104265
9,alonzo gee,0.137681159420290,0.460784313725490,0.298797409805735,0.478260869565217


In [ ]:
Player_Stats_1415.info()

In [11]:
Player_Stats_1415 = Player_Stats_1415[pd.notnull(Player_Stats_1415["conditional_prob"])]

Again, as in the classroom, we can first check which players have the highest conditional probability, i.e., more likely to have hot hand.

Let's sort the data by conditional probability.

In [12]:
Player_Stats_1415.sort_values(by=['conditional_prob'], ascending=[False]).head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
66,deandre jordan,0.422391857506361,0.688821752265861,0.613209231730726,0.712468193384224
270,tyson chandler,0.373529411764706,0.659649122807018,0.566254693366708,0.676470588235294
188,mason plumlee,0.315136476426799,0.601744186046512,0.523705062274487,0.585607940446650
137,jonas valanciunas,0.294238683127572,0.563380281690141,0.522273662551440,0.563786008230453
19,anthony davis,0.281807372175981,0.544191919191919,0.517845565576280,0.543400713436385
82,ed davis,0.291428571428571,0.570446735395189,0.510877796901893,0.602857142857143
109,james johnson,0.302250803858521,0.610687022900763,0.494935691318328,0.614147909967846
208,nikola vucevic,0.261640798226164,0.531914893617021,0.491884700665189,0.532150776053215
235,rudy gobert,0.295880149812734,0.602870813397129,0.490785327863980,0.625468164794007
71,derrick favors,0.264401772525849,0.539452495974235,0.490129852950903,0.545051698670606


Comparing the "conditional_prob" variable and the "average_hit" variable, some players have a slightly higher conditional probability but some also have a lower conditional probability.

We can sort the data by the value of difference between conditional and unconditional probabilities.

In [13]:
Player_Stats_1415['diff_prob'] = Player_Stats_1415['conditional_prob'] - Player_Stats_1415['average_hit']

Player_Stats_1415.sort_values(by=['diff_prob'], ascending=[False]).head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit,diff_prob
200,nate robinson,0.128712871287129,0.343195266272189,0.375042676681461,0.341584158415842,0.033458518265620
226,reggie jackson,0.182094081942337,0.423841059602649,0.429628224582701,0.420333839150228,0.009294385432473
114,jason smith,0.210648148148148,0.466487935656836,0.451561834823329,0.451388888888889,0.000172945934440
72,derrick rose,0.167958656330749,0.413461538461538,0.406225587404603,0.406976744186047,-0.000751156781443
83,elfrid payton,0.168750000000000,0.413875598086124,0.407731213872832,0.412500000000000,-0.004768786127168
25,ben mclemore,0.195876288659794,0.453154875717017,0.432250206620558,0.438144329896907,-0.005894123276350
273,vince carter,0.106007067137809,0.329166666666667,0.322046786241446,0.332155477031802,-0.010108690790356
50,cj mccollum,0.160919540229885,0.423357664233577,0.380103051922315,0.390804597701150,-0.010701545778835
229,robert covington,0.154901960784314,0.402597402597403,0.384756483238457,0.396078431372549,-0.011321948134092
23,avery bradley,0.182781456953642,0.436154949784792,0.419074590449634,0.430463576158940,-0.011388985709306


##### Interpretation of Results:
When we sort the data by the value of the difference between conditional and unconditional probabilities, we can see that Nate Robinson has the highest difference between the two probabilities, at 3%. Reggie Jackson follows in second place at 0.9%  

##### Calculate summary statistics for the probability for a player to make a shot (“average_hit”) and the conditional probability for a player to make a shot given that he made the previous one (“conditional_prob”) and the probability of players making consecutive shots (“conse_shot_hit”).

In [ ]:
Player_Stats_1415[['average_hit', 'conditional_prob', 'conse_shot_hit']].describe()

### T-test for statistical significance on the difference

More rigorously, we can use a t-test to test if the players’ probability of hitting the goal is statistically significantly different than their conditional probability. 

We need to choose a significance level before we perform the test. If the test produces a p-value less than the chosen significance level, then we say that there is a statistically significant difference between the two probabilities; otherwise, we fail to find evidence to support that the two probabilities are statistically significantly different from each other. 

The most commonly used significance level is 0.05.

#### To perform a t-test, we need to import a new library, "scipy.stats."

We can use the ttest_ind() function to calculate the test statistics.

In [14]:
sp.stats.ttest_ind(Player_Stats_1415['conditional_prob'], Player_Stats_1415['average_hit'])

Ttest_indResult(statistic=-13.885932802814914, pvalue=6.925846314604593e-38)

The first number is the t-statistics and the second number is the p-value.

Note that the p-value for the t test is 6.925846314604593e-38, which is way smaller than the conventional significance level 0.05. Thus the conditional probability is statistically significantly different than the average success rate. In other words, in the analysis of conditional probability, we seem to find evidence to support the "hot hand".

##### Autocorrelation Coefficient
We can calculate the autocorrelation coefficient by calculating the correlation coefficient between the “current_shot_hit” variable and the “lag_shot_hit” variable.

**Note: in python, you could use “autocorr(lag=1)” to calculate first order autocorrelation coefficient. This command is not very useful in our case since we want to look at the autocorrelation coefficient within each game. Using the built-in autocorrelation coefficient function in python, we will be treating the last shot from the previous game and the first shot of the subsequent game as a pair.**

In [15]:
Shotlog_1415['current_shot_hit'].corr(Shotlog_1415['lag_shot_hit'])

-0.010502388301693177

_As we can see, the autocorrelation coefficient is negative and the magnitude is quite small and close to zero._

Since some players may have “hot hand”, and hence strong correlation between outcomes of adjacent shots, while some may not.  We can also calculate autocorrelation coefficient for each player.

In [16]:
Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().head(10)

current_shot_hit       lag_shot_hit
shoot_player                                                          
aaron brooks    current_shot_hit  1.000000000000000  0.002555783853533
                lag_shot_hit      0.002555783853533  1.000000000000000
aaron gordon    current_shot_hit  1.000000000000000 -0.071283243564025
                lag_shot_hit     -0.071283243564025  1.000000000000000
al farouq aminu current_shot_hit  1.000000000000000  0.029513112934917
                lag_shot_hit      0.029513112934917  1.000000000000000
al horford      current_shot_hit  1.000000000000000 -0.008396493626859
                lag_shot_hit     -0.008396493626859  1.000000000000000
al jefferson    current_shot_hit  1.000000000000000 -0.038884873208338
                lag_shot_hit     -0.038884873208338  1.000000000000000

#### We may not want to print out a 2 by 2 matrix for every player. We can use the "unstack()" command to reshape the data.



In [17]:
Autocorr_Hit_1415=Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit_1415.head()

,shoot_player,current_shot_hit
,,lag_shot_hit
0,aaron brooks,0.002555783853533
1,aaron gordon,-0.071283243564025
2,al farouq aminu,0.029513112934917
3,al horford,-0.008396493626859
4,al jefferson,-0.038884873208338


Notice that we still have two levels of variable names. 

#### We can use the "get_level_values" command to reset the variable name to the first level (index 0).

In [18]:
Autocorr_Hit_1415.columns=Autocorr_Hit_1415.columns.get_level_values(0)
Autocorr_Hit_1415.head()

,shoot_player,current_shot_hit
0,aaron brooks,0.002555783853533
1,aaron gordon,-0.071283243564025
2,al farouq aminu,0.029513112934917
3,al horford,-0.008396493626859
4,al jefferson,-0.038884873208338


Let's rename the variable capturing autocorrelation coefficient.

In [19]:
Autocorr_Hit_1415.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)
Autocorr_Hit_1415.head()

,shoot_player,autocorr
0,aaron brooks,0.002555783853533
1,aaron gordon,-0.071283243564025
2,al farouq aminu,0.029513112934917
3,al horford,-0.008396493626859
4,al jefferson,-0.038884873208338


In [20]:
Autocorr_Hit_1415.sort_values('autocorr',ascending=False).head(10)

,shoot_player,autocorr
131,joey dorsey,0.334251608718693
54,cole aldrich,0.174666121980616
200,nate robinson,0.122106749829680
267,tyler hansbrough,0.120607558343718
7,alex len,0.118460586521278
50,cj mccollum,0.115948705896904
114,jason smith,0.105903078611449
190,matt bonner,0.098576517571810
143,jusuf nurkic,0.097464741091723
195,mike miller,0.089366011125439


## Part 3 - Regression Analyses

In [ ]:
print(Shotlog_1415.columns.tolist())

In [ ]:
Shotlog_1415.info()

In [21]:
Shotlog_1415['quarter'] = Shotlog_1415.quarter.astype('category')

In [ ]:
Shotlog_1415.info()

In [22]:
Reg1 = sm.ols(formula = 'lagerror ~ shot_dist+dribbles+touch_time+C(points)+home_away+C(quarter)+\
              shoot_player+closest_defender+closest_def_dist',
              data= Shotlog_1415).fit()
print(Reg1.summary())

                            OLS Regression Results                            
Dep. Variable:               lagerror   R-squared:                -1748158.302
Model:                            OLS   Adj. R-squared:           -1759981.796
Method:                 Least Squares   F-statistic:                    -147.9
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               1.00
Time:                        23:26:51   Log-Likelihood:            -8.9899e+05
No. Observations:              113726   AIC:                         1.800e+06
Df Residuals:                  112961   BIC:                         1.807e+06
Df Model:                         764                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

### Weighted least squares regression
As we have seen, some players had a lot of shot per game while some just had a few. Different players may have different variations in their success rate in the shots. We can run a weighted least squared regression to address this problem. 

Weighted least squares estimation weights the observations proportional to the reciprocal of the error variance of the observation. Thus weighted least squares can overcome the issue of non-constant variance.

#### We can use the “sm.wls” command to run the weighted least square regression weighted by the number of shot per game (weight=1/shot_per_game).

In [22]:
Reg2 = sm.wls(formula = 'lagerror ~ shot_dist+dribbles+touch_time+C(points)+home_away+C(quarter)+\
              shoot_player+closest_defender+closest_def_dist',
              weights=1/Shotlog_1415['shot_per_game'] , data= Shotlog_1415).fit()
print(Reg2.summary())

                            WLS Regression Results                            
Dep. Variable:               lagerror   R-squared:                 -497732.778
Model:                            WLS   Adj. R-squared:            -501099.149
Method:                 Least Squares   F-statistic:                    -147.9
Date:                Fri, 17 Feb 2023   Prob (F-statistic):               1.00
Time:                        00:09:39   Log-Likelihood:            -8.3675e+05
No. Observations:              113726   AIC:                         1.675e+06
Df Residuals:                  112961   BIC:                         1.682e+06
Df Model:                         764                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------